# SimpleSim Non-Holonomic Navigation Challenge

This notebook attempts to train an agent solve a simplesim non-holonomic driving navigation problem with 1 target with random spawn location.
This time, however we are teaching the agent to dwell at the goal as well instead of simply ending the episode.


## Install Dependencies and Stable Baselines3 Using Pip

In [ ]:
# !pip install "stable-baselines3[extra]>=2.0.0a4"

### Setup Tensorboard Logging

In [ ]:
# # Clear any logs from previous runs
# !rm -rf ./logs/

# Load the TensorBoard notebook extension
%load_ext tensorboard

##  Custom Gym Envs

Below are a couple of simpler lower order gridworld type gym environments that can be used as testing and debugging examples ,as well as our main SimpleSIm non-holonomic driving environment (which is imported from the seperate source files env.py and env_gym.py)

In [ ]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces

# Import our main environment
from env_gym import SimpleSimGym

### Validate the environment

Stable Baselines3 provides a [helper](https://stable-baselines3.readthedocs.io/en/master/common/env_checker.html) to check that your environment follows the Gym interface. It also optionally checks that the environment is compatible with Stable-Baselines (and emits warning if necessary).

In [ ]:
from stable_baselines3.common.env_checker import check_env

env = SimpleSimGym(max_budget=500, max_targets=3, num_classes=10, player_fov=60)

# If the environment don't follow the interface, an error will be thrown
check_env(env, warn=True)

## Import Auto Saving of Best Model Callback

In [ ]:
from utils import SaveOnBestTrainingRewardCallback

## Train The Model

In [ ]:
from stable_baselines3 import PPO, SAC, TD3
from stable_baselines3.common.env_util import make_vec_env

# Environment Parameters
MAX_BUDGET = 400
MAX_TARGETS = 5
NUM_CLASSES = 10
PLAYER_FOV = 30
RENDER_MODE = "rgb_array"
ACTION_FORMAT = "continuous"

config = {
    "policy": 'MlpPolicy',
    "total_timesteps": 1_000_000,
    "logdir": "logs/",
    "savedir": "saved_models/",
}

# Create log dir
os.makedirs(config["logdir"], exist_ok=True)

# Create save dir
os.makedirs(config["savedir"], exist_ok=True)

# Show Tensorboard Logs
Visualise the live logs on tensorboard as we train

In [ ]:
# Open Tensorboard Logging
%tensorboard --logdir logs/ --reload_interval 30

### Train PPO

In [ ]:
# # Instantiate and wrap the env
# env_ppo = make_vec_env(SimpleSimGym, 
#                    n_envs=1, 
#                    monitor_dir=config["logdir"], 
#                    env_kwargs=dict(
#                        max_budget=MAX_BUDGET, 
#                        max_targets=MAX_TARGETS, 
#                        num_classes=NUM_CLASSES, 
#                        player_fov=PLAYER_FOV, 
#                        render_mode=RENDER_MODE, 
#                        action_format=ACTION_FORMAT))

# # Setup callbacks
# ppo_save_callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=config["logdir"], save_dir=config["savedir"], verbose=0)
# # ppo_save_callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=config["logdir"], save_dir=config["savedir"], model_name="ppo", verbose=0)

# # # Load the model from checkpoint 2
# # model_ppo = PPO.load(f"{config['savedir']}/MlpPolicy_PPO_step2000000", env_ppo)

# # Create the agent
# model_ppo = PPO(config["policy"], env_ppo, tensorboard_log=config["logdir"], verbose=0)

# # # Train the agent
# # model_ppo.learn(config["total_timesteps"], tb_log_name="PPO", callback=ppo_save_callback, progress_bar=True)
# # model_ppo.save(f"{config['savedir']}/{config['policy']}_PPO")

# # Train in tranches
# num_tranches = 10
# for i in range(1, num_tranches+1):
#     print(f"Tranch {i}/{num_tranches}:")
#     model_ppo.learn(config["total_timesteps"]//num_tranches, tb_log_name="PPO", callback=ppo_save_callback, progress_bar=True, reset_num_timesteps=False)
#     model_ppo.save(f"{config['savedir']}/{config['policy']}_PPO_step{i * (config['total_timesteps']//num_tranches)}")

### Train SAC

In [ ]:
# Instantiate and wrap the env
env_sac = make_vec_env(SimpleSimGym, 
                   n_envs=1, 
                   monitor_dir=config["logdir"]+"sac", 
                   env_kwargs=dict(
                       max_budget=MAX_BUDGET, 
                       max_targets=MAX_TARGETS, 
                       num_classes=NUM_CLASSES, 
                       player_fov=PLAYER_FOV, 
                       render_mode=RENDER_MODE, 
                       action_format=ACTION_FORMAT))

# Setup callbacks
auto_save_callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=config["logdir"], save_dir=config["savedir"], model_name="sac", verbose=0)

# # Load the model from checkpoint 6M
# model_sac = SAC.load(f"{config['savedir']}/MlpPolicy_SAC_step8000000", env_sac)
# Create the agent
model_sac = SAC(config["policy"], env_sac, tensorboard_log=config["logdir"], verbose=0)

# # # Train the agent
# # model_sac.learn(config["total_timesteps"], tb_log_name="SAC", callback=auto_save_callback, progress_bar=True)
# # model_sac.save(f"{config['savedir']}/{config['policy']}_SAC")

# Train in tranches
times_trained = 0
num_tranches = 10
tranches = num_tranches*(1+times_trained)

for i in range((num_tranches * times_trained)+1, tranches+1):
    print(f"^ Tranch {i}/{tranches}")
    if i == 1:
        model_sac.learn(config["total_timesteps"]//tranches, tb_log_name="SAC", callback=auto_save_callback, progress_bar=True, reset_num_timesteps=True)
    else:
        model_sac.learn(config["total_timesteps"]//tranches, tb_log_name="SAC", callback=auto_save_callback, progress_bar=True, reset_num_timesteps=False)
    model_sac.save(f"{config['savedir']}/{config['policy']}_SAC_step{i * (config['total_timesteps']//tranches)}")

### Continue Training or Run Dupliacte Experiments?

This cell can be used to either continue training on an existing model (use `reset_num_timesteps=False`) or to run additional duplicate experiments training from scratch to test training consistency

In [ ]:
# model_sac.learn(config["total_timesteps"], tb_log_name="SAC", callback=auto_save_callback, progress_bar=True, reset_num_timesteps=False)
# model_sac.save(f"{config['savedir']}/{config['policy']}_SAC_pt2")

### Load Model?

In [ ]:
# Load the best model
# model_sac = SAC.load(f"{config['savedir']}/MlpPolicy_SAC_step4000000")
# model_ppo = PPO.load(f"{config['savedir']}/MlpPolicy_PPO_step4000000")

best_sac = SAC.load(f"{config['savedir']}/best_sac")
# best_ppo = PPO.load(f"{config['savedir']}/best_ppo")

### Check Performance

Check if the policy can consistently succeed in the environment over multilpe episodes.

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

# Instantiate the eval env
eval_env = make_vec_env(SimpleSimGym, 
                   n_envs=1, 
                   # monitor_dir=config["logdir"], 
                   env_kwargs=dict(
                       max_budget=MAX_BUDGET, 
                       max_targets=MAX_TARGETS, 
                       num_classes=NUM_CLASSES, 
                       player_fov=PLAYER_FOV, 
                       render_mode=RENDER_MODE, 
                       action_format=ACTION_FORMAT
                   )
                  )

# Check performance of best vs last model
models = {"last_sac": model_sac, "best_sac": best_sac}#, "last_ppo": model_ppo, "best_ppo": best_ppo}
# models = {"best_sac": best_sac, "best_ppo": best_ppo}

for key in models.keys():
    # Reset the eval env
    eval_env.reset()
    # Test average reward over multiple episodes
    mean_reward, std_reward = evaluate_policy(models[key], eval_env, n_eval_episodes=50)
    print(f"MODEL TYPE: {key}")
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}\n")

### Visualize Trained Agent with Video


In [ ]:
from utils import record_video, show_videos

record_video(eval_env, model_sac, video_length=500*3, prefix="jupyter-sac-last-simplesim")
show_videos("videos", prefix="jupyter-sac-last")

record_video(eval_env, best_sac, video_length=500*3, prefix="jupyter-sac-best-simplesim")
show_videos("videos", prefix="jupyter-sac-best")

In [ ]:
# record_video(eval_env, model_ppo, video_length=500*3, prefix="jupyter-ppo-last-simplesim")
# show_videos("videos", prefix="jupyter-ppo-last")

# record_video(eval_env, best_ppo, video_length=500*3, prefix="jupyter-ppo-best-simplesim")
# show_videos("videos", prefix="jupyter-ppo-best")